# Homework 1 - Vector Space Model

## Import package

In [ ]:
import math
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity

## Reading file
讀取 Document 和 Query，並將內容以切割成文章單字列表

In [ ]:
doc_list_filename = 'data/doc_list.txt'  # doc_list 檔案路徑
query_list_filename = 'data/query_list.txt'  # query_list 檔案路徑
doc_path = 'data/docs/'  # document 檔案資料夾路徑
query_path = 'data/queries/'  # query 檔案資料夾路徑

In [ ]:
doc_list = pd.read_table(doc_list_filename, header=None)[0].tolist()
query_list = pd.read_table(query_list_filename, header=None)[0].tolist()

In [ ]:
def read_and_split(file_path, file_list, description):
    text_list = []
    text_split_list = []
    pbar = tqdm(file_list)  # 進度條
    pbar.set_description('Reading %s' % description)
    for file in pbar:
        filename = file_path + str(file) + '.txt'
        try:
            text = pd.read_table(filename, header=None)[0][0]  # 只讀取檔案的第一行
        except:
            text = ''
        text_list.append(text)  # 檔案完整內容
        text_split_list.append(text.split())  # 檔案內容切成單字列表
    return text_list, text_split_list

In [ ]:
doc_text, doc_text_split = read_and_split(doc_path ,doc_list, 'doc')
query_text, query_text_split = read_and_split(query_path ,query_list, 'query')

## Index term
將 Document 與 Query 各文章單字列表合併，去除重複得到 Index term。

In [ ]:
def list_remove_duplicates(data_list):
    t_list = []
    for data in data_list:
        t_list += data
    t_dict = {}.fromkeys(t_list)  # 利用 Dictionary key 不重複的特性，取得 Index Term
    return list(t_dict)  # 將 Dictionary 轉成 List 回傳

In [ ]:
index_term = list_remove_duplicates(doc_text_split + query_text_split)

## Vector Space Model

### Term Frequency
根據剛剛的 Index Term 列表，統計各 Document & Query 出現每個在 Index Term 中 Word 的次數。

+ ${Term\;Frequency}=tf_{ij}$

In [ ]:
def term_frequency(index_term, document):
    doc_matrix = []
    pbar = tqdm(document)  # 進度條
    pbar.set_description('TF')
    for doc in pbar:
        doc_vector = []
        for word in index_term:  # 根據 Index Term 中每個 Word
            doc_vector.append(doc.count(word))  # 計數該 Word 在這個 Document 出現幾次
        doc_matrix.append(doc_vector)
    return np.array(doc_matrix)  # 將 2D list 轉成 Numpy.array 回傳

In [ ]:
# 由於計算Term Frequency非常耗時，
# 因此可將這兩個 matrix 存起來，方便重複使用，直接對此 Matrix 套入公式調整 TF Matrix
doc_tf_matrix = term_frequency(index_term, doc_text_split)
np.save('doc_tf_matrix', doc_tf_matrix)

In [ ]:
query_tf_matrix = term_frequency(index_term, query_text_split)
np.save('query_tf_matrix', query_tf_matrix)

### Inverse Document Frequency
目的是想知道每個 Word 的「獨特性」。

1. 我們首先計算每個 Word 在出現在各 Document 的頻率，得到 Document Frequency Matrix。

2. 再將 Document Frequency Matrix 轉成 Inverse Document Frequency Matrix。

+ ${Document\;Frequency}=\frac{N}{n_{i}}$
+ ${Inverse\;Document\;Frequency}=log(1+\frac{N+1}{n_{i}+1})$

In [ ]:
def document_frequency(index_term, document):
    df_list = []
    num_of_doc = len(document)
    pbar = tqdm(index_term)
    pbar.set_description('DF')
    for word in pbar:
        doc_count = 0
        for doc in document:
            if word in doc:
                doc_count += 1
        df_list.append(doc_count)  # df weight
    return np.array(df_list)

In [ ]:
doc_df_matrix = document_frequency(index_term, doc_text_split)

In [ ]:
# 分子分母皆加一，避免除零錯誤
doc_idf_matrix = 1 + np.log((len(doc_text_split)+1)/(doc_df_matrix+1))

In [ ]:
# 由於計算Term Frequency也相當耗時，
# 因此可將可 matrix 存起來，方便重複使用，直接對此 Matrix 套入公式調整 IDF Matrix
np.save('doc_df_matrix', doc_df_matrix)

### TF-IDF
將 TF Matrix 和 IDF Matrix 相乘，即可得到 TF-IDF Matrix，每個 Document & Query 皆有一個代表它的向量。
+ $𝑇𝐹−𝐼𝐷𝐹_{𝑖,𝑗}=𝑡𝑓_{𝑖,𝑗}\times𝑖𝑑𝑓_{𝑖}$

In [ ]:
doc_tf_matrix = np.load('doc_tf_matrix.npy')
query_tf_matrix = np.load('query_tf_matrix.npy')
doc_df_matrix = np.load('doc_df_matrix.npy')

In [ ]:
doc_tf_idf_matrix = doc_tf_matrix * doc_idf_matrix
query_tf_idf_matrix = query_tf_matrix * doc_idf_matrix

### Cosine similarity
計算餘弦相似性，得出每個 Query 與各 Document 的相似程度。

In [ ]:
cos_matrix = cosine_similarity(query_tf_idf_matrix, doc_tf_idf_matrix)

### Rank
1. 根據剛剛的 Cosine similarity Matrix，可以把每個 Query 與所有 Document 的相似程度做排名，並把排名結果以 Document 檔名依序列出，存成一個 Retrieved Documents List。
2. 把 Query List 和 Retrieved Documents List 建成一個 DatafFrame，輸出成 CSV。

In [ ]:
retrieved_documents_list = []

pbar = tqdm(range(cos_matrix.shape[0]))
pbar.set_description('Ranking')
for i in pbar:
    # np.argsort(np.argsort(Vector)) 可得到該 Value 在此 Vector 的名次(越大名次越高)
    retrie_doc_value_dict = dict(zip(doc_list, np.argsort(np.argsort(cos_matrix[i]))))
    # 將 (key, value) 根據 Value 進行排序，輸出 key
    retrie_doc_sort_list = sorted(retrie_doc_value_dict.items(),
                                                                  key=lambda retrie_doc_value_dict:retrie_doc_value_dict[1], 
                                                                  reverse = True)
    # 將每個 key 以空格分隔輸出成 String 放至 Retrieved Documents List
    retrieved_documents_list.append(' '.join([doc[0] for doc in retrie_doc_sort_list]))

In [ ]:
# 存成 DataFrame 
submission_df = pd.DataFrame(data={'Query': query_list,
                                                                               'RetrievedDocuments': retrieved_documents_list})
submission_df.head()

In [ ]:
# 輸出成 CSV
submission_df.to_csv('submission.csv', index=False)